In [1]:
using BenchmarkTools
using Distances
using NNDescent
using NNDescent: brute_knn, NNTuple

In [2]:
recall(nn, true_nn) = sum(_recall(nn[:,i], true_nn[:,i]) for i in 1:size(nn,2))/size(nn,2)
_recall(π, πₜ) = length(intersect(π, πₜ))/length(πₜ)

_recall (generic function with 1 method)

In [3]:
data = [rand(20) for _ in 1:10000]
queries = [rand(20) for _ in 1:1000]

knn_graph = DescentGraph(data, 10)
nn = getindex.(knn_graph.graph, 1)

10×10000 Array{Int64,2}:
 5858  4213  5344  2726  6111  1531  …  9162  8457  2811  8217   961  2852
 2651  3885  5252  8531  4349   768     9269  6608  8955   699  3493  7633
 9896  6255  9265  2780  9349  2105     8864  4980  9959  5715  1183  1906
 1012  9500  6542  9244  3399    25     1970  6748   110  1266  9053   103
 9810  8877   910  6759  9008  5514     8650  8816  1141  9009  7319  2109
  256  3955  7800  3219  7282  2670  …  7916  6482  2810  4672  6799  8786
 7650  8655  4194  8349  6824  8593      749  2687  8419  3450  8376  6525
 5318   275  2067  6779  9703  8691     1325  4268  5357  3710   582  4700
 8003  6591  9953  9198   716  6711     1242  7699  6067  1633  8473  9152
 5694  7495  8672  9136  4409  7607     4723  6471  1149  1341   900  6086

In [4]:
brute_graph = brute_knn(data, Euclidean(), 10)
true_nn = getindex.(brute_graph, 1)

10×10000 Array{Int64,2}:
 5858  4213  5344  2726  6111  1531  …  9162  8457  2811  8217   961  2852
 2651  3885  5252  9432  4349   768     9269  6608  8955   699  3493  7633
 9896  6255  9265  8531  7761  2105     8864  4980  9959  5715  2982  1906
 1012  9500  6542  2780  9349  2558     1970  6748   110  1266  1183   103
 9810  8877   910  9244   242    25     8650  8816  1141  9009  1376  2109
 6948  3955  7533  6759  3399  5514  …  7916  6482  2810  4672  7850  8786
  256  8655  7800  3219  3940  2670      749  2687  8419  3450  9053  6525
 7650   275  4194  8349  9008  8593     1325  4268  5357  3710  7656  4700
 5318  6591  2067  6779  3080   565     1242  7699  6067  1996  7319  9152
 8003  7495  9953  5068    86  8691     4723  6471  1149  1633  6799  6086

In [5]:
recall(nn, true_nn)

0.906239999999954

In [6]:
function brute_search(data, queries, k, metric)
    np = length(data)
    nq = length(queries)
    distances = Matrix{NNTuple{Int, Float64}}(undef, np, nq)
    
    @inbounds @fastmath for i = 1:nq, j = 1:np
        d = evaluate(metric, queries[i], data[j])
        distances[j, i] = NNTuple(j, d)
    end
    
    knn_graph_tuples = sort(distances, dims=1)[1:k, 1:end]
    knn_graph = Matrix{Tuple{Int, Float64}}(undef, k, nq)
    for i = 1:k, j = 1:nq
        knn_graph[i, j] = (knn_graph_tuples[i, j].idx, knn_graph_tuples[i, j].dist)
    end
    return getindex.(knn_graph, 1), getindex.(knn_graph, 2)
end

brute_search (generic function with 1 method)

In [7]:
true_idx, true_dist = brute_search(data, queries, 5, Euclidean())
true_idx

5×1000 Array{Int64,2}:
 7720  4543  7704  5313  1598  2360  …  4782  7889  6887  2836  6846  4412
 5613  4890   847  9327  1449   103     6577  9364  3591  2885  9168  6752
 6460  4605  9489  6429  6387  6893     7561  3020  3690  6621    91  7504
 5564  8875   510   303  9052  9192     7800  5924  8152  3598  8206  3056
 1493  6136  2798  4060  5425  1515     4294   104  7840  5555  6660   654

In [8]:
idx, dist = search(knn_graph, queries, 5, 20)
idx

5×1000 Array{Int64,2}:
 7720  4543  7704  5313  1598  2360  …  4782  7889  6887  2836  6846  4412
 5613  4890   847  9327  1449   103     6577  9364  3591  2885  9168  6752
 6460  4605  9489  6429  6387  6893     7561  3020  3690  6621    91  7504
 5564  8875   510   303  9052  9192     7800   104  8152  3598  8206  3056
 1493  6136  2798  4060  5425  1515     4294  4643  7840  5555  6660  2858

In [9]:
recall(idx, true_idx)

0.9307999999999961

In [10]:
@belapsed search(knn_graph, queries, 5, 20)

0.290359395